In [1]:
!git clone https://github.com/kimdonggyu2008/tacotron_tts_practice
%cd tacotron_tts_practice
!git submodule init; git submodule update

Cloning into 'tacotron_tts_practice'...
remote: Enumerating objects: 174, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 174 (delta 34), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (174/174), 20.49 MiB | 13.13 MiB/s, done.
Resolving deltas: 100% (34/34), done.
/kaggle/working/tacotron_tts_practice


In [2]:
import pandas as pd


metadata = pd.read_csv("../../input/johnoliver/metadata.csv")
metadata = metadata[metadata['include']]

total = len(metadata)
split = 0.9
train = int(total * split)

train_metadata = metadata[:train]
val_metadata = metadata[train:]

with open('/kaggle/working/tacotron_tts_practice/filelists/audio_text_train_filelist.txt', 'w') as f:
    for _, (index, _, _, _, text, _) in train_metadata.iterrows():
        filepath = '../../input/johnoliver/wav/%05d.wav' % index
        f.write(filepath + '|' + text + '\n')

with open('/kaggle/working/tacotron_tts_practice/filelists/audio_text_test_filelist.txt', 'w') as f:
    for _, (index, _, _, _, text, _) in val_metadata.iterrows():
        filepath = '../../input/johnoliver/wav/%05d.wav' % index
        f.write(filepath + '|' + text + '\n')

In [3]:
df = pd.read_csv("/kaggle/working/tacotron_tts_practice/filelists/ljs_audio_text_train_filelist.txt", sep="|", names=["path", "text"])

In [4]:
with open("/kaggle/working/tacotron_tts_practice/filelists/ljs_audio_text_train_filelist.txt", "r") as f:
    lines = f.readlines()
with open("/kaggle/working/tacotron_tts_practice/filelists/ljs_audio_text_train_filelist.txt", "w") as f:
    for line in lines:
        line = line.replace('DUMMY', '/kaggle/input/the-lj-speech-dataset/LJSpeech-1.1/wavs')
        f.write(line)

In [5]:
!mv /kaggle/working/tacotron_tts_practice/filelists/ljs_audio_text_train_filelist.txt /kaggle/working/tacotron_tts_practice/filelists/audio_text_train_filelist.txt

In [6]:
!mv /kaggle/working/tacotron_tts_practice/filelists/ljs_audio_text_test_filelist.txt /kaggle/working/tacotron_tts_practice/filelists/audio_text_test_filelist.txt

In [7]:
!ls tacotron_tts_practice/filelists

ls: cannot access 'tacotron_tts_practice/filelists': No such file or directory


In [8]:
!pip install tensorflow==2.17
!pip install unidecode
!pip install inflect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 49.0 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.5.26
    Uninstalling flatbuffers-23.5.26:
      Successfully uninstalled flatbuffers-23.5.26
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.1
    Uninstalling tensorboard-2.15.1:
      Successfully uninstalled tensorboard-2.15.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the 

In [9]:
!python --version

Python 3.10.13


In [10]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1c5ZTuT7J08wLUoVZ2KkUs_VdZuJ86ZqA

Downloading...
From (original): https://drive.google.com/uc?id=1c5ZTuT7J08wLUoVZ2KkUs_VdZuJ86ZqA
From (redirected): https://drive.google.com/uc?id=1c5ZTuT7J08wLUoVZ2KkUs_VdZuJ86ZqA&confirm=t&uuid=d1c3e4fc-b4db-472a-b139-59832eb9b11b
To: /kaggle/working/tacotron_tts_practice/tacotron2_statedict.pt
100%|█████████████████████████████████████████| 113M/113M [00:01<00:00, 102MB/s]


In [11]:
!apt-get update --fix-missing
!apt-get install sox -y
!soxi /kaggle/input/the-lj-speech-dataset/LJSpeech-1.1/wavs/LJ001-0001.wav

Get:1 https://packages.cloud.google.com/apt gcsfuse-focal InRelease [1225 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 https://packages.cloud.google.com/apt cloud-sdk InRelease [1616 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1581 B]
Hit:5 https://packages.cloud.google.com/apt google-fast-socket InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Get:9 https://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [24.8 kB]
Get:10 https://packages.cloud.google.com/apt cloud-sdk/main all Packages [1480 kB]
Get:11 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [3107 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1557 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-update

In [12]:
#!python train.py --output_directory=/kaggle/working/tacotron_tts_practice --log_directory=/kaggle/working/tacotron_tts_practice -c tacotron2_statedict.pt --warm_start 

In [13]:
!python train.py --output_directory=outdir --log_directory=logdir -c tacotron2_statedict.pt --warm_start --n_gpus=4 --rank=1 --group_name='group_name' --hparams sampling_rate=22050:iters_per_checkpoint=100

2024-07-20 07:48:01.818343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-20 07:48:01.842765: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-20 07:48:01.850032: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
FP16 Run: False
Dynamic Loss Scaling: True
Distributed Run: False
cuDNN Enabled: True
cuDNN Benchmark: False
Warm starting model from checkpoint 'tacotron2_statedict.pt'
Epoch: 0
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this wil